# Классификация одежды.

Необходимо построить модель, которая по фотографии определит тип предмета гардероба. В нашем распоряжении набор фотографий вещей из датасета [Fashion MNIST](https://www.kaggle.com/datasets/zalando-research/fashionmnist).

Последовательность решения задачи:
1. Анализ данных;
2. Подготовка данных;
3. Создание модели и её обучение;
4. Вывод и оценка качества модели.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.applications.resnet import ResNet50

In [2]:
PATH = '/kaggle/input/fashionmnist/'

## Анализ данных

Загрузим и проанализируем данные.

In [3]:
# Загрузка данных
df_train = pd.read_csv(PATH + 'fashion-mnist_train.csv')
df_test = pd.read_csv(PATH + 'fashion-mnist_test.csv')

In [4]:
def data_info(df):
    """Изучение датасета"""
    display(df.head())
    display(df.info())
    # Проверка дизбаланса классов
    df['label'].hist()
    print('Количество дубликатов =', df.duplicated().sum())
    

In [5]:
# Анализ трениоровочных данных
data_info(df_train)

In [6]:
# Анализ тестовых данных
data_info(df_test)

In [7]:
# Надвания классов вещей
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [8]:
# Выделим по одному изображению каждого класса
one_category = df_train.groupby('label').head(1)
one_category = one_category.set_index('label')

# Вывод на экран изображений
fig, axes = plt.subplots(nrows=1, ncols=10, figsize=(20,7))

n = 0
for ax in axes.flat:
    ax.set(title=(class_names[n]))
    image = one_category.loc[n,:].values.reshape(28, 28, 1)
    ax.imshow(image, cmap=plt.cm.binary)
    n += 1
plt.show()

**Вывод**

1. Набор данных содержит 60000 изображений в тренировочном наборе и 10000 изображений в тестовом наборе данных. Такого количества должно быть достаточно для обучения модели.
2. Вещи предствалены 10 классов, причём их количество по классам одинаково. Таким образом, дизбаланса классов не наблюдается.
3. Изображеняи имеют размер 28Х28 пиксклей и один канал цвета. В датасети они представлены в виде списка, поэтому на этапе предобработки, их необходимо привести в двумерный вид.
4. Изображения предобработаны, и не требуют дальнейшего преобразования.
5. В тренировочном наборе данных присутствует 43 дубликата. В целом, такое небольшое количество повторяющехся изображений не должно сильно сказаться на качестве обучения модели, думаю лучше будет от них избавится.


## Подготовка данных и обучение модели

1. Избавимся от дубликатов.
2. Создадим функции `load_train` и `load_test` по подготовке данных тренировочного и тестового набора данных. В них произведём преобразование данных из одномерного списка в двумерное изображение и нормализуем их.
3. Создадим функцию `create_model_LeNet`, которая будет создавать и компилировать модель. Применим нейросетевую модель с архитектурой LeNet. Оптимизатор и его параметры выберем опытным путём.
4. Создадим функцию `train_model`, которая будет отвечать за обучение модели.
5. Обучим модель.

In [9]:
# Удалим дубликаты
df_train = df_train.drop_duplicates()

In [10]:
def load_train(df_train):
    """ Принимаем датафрейм с тренировочными данными.
        Возвращаем преобразованные и нормализованные признаки и целевой признак
    """
    features_train = df_train.drop('label', axis=1).values
    target_train = df_train['label']
    features_train = features_train.reshape(-1, 28, 28, 1) / 255.0
    return features_train, target_train

In [11]:
def load_test(df_test):
    """ Принимаем датафрейм с тестовыми данными.
        Возвращаем преобразованные и нормализованные признаки и целевой признак
    """
    features_test = df_test.drop('label', axis=1).values
    target_test = df_test['label']
    features_test = features_test.reshape(-1, 28, 28, 1) / 255.0
    return features_test, target_test

In [12]:
def create_model(input_shape, optimizer=Adam(learning_rate=0.001)):
    """ Принимаем размер входных данных и оптимизатор.
        Возвращаем созданную модель
    """
    model = Sequential()
    model.add(Conv2D(kernel_size=(5, 5), filters=6, padding='same', activation='relu', input_shape=input_shape))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Conv2D(kernel_size=(5, 5), filters=16, padding='valid', activation='relu', strides=1))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=120, activation='relu'))
    model.add(Dense(units=84, activation='relu'))
    model.add(Dense(units=10, activation='softmax'))
   
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
    model.summary()
    
    return model


In [13]:
def train_model(model, train_data, test_data, batch_size=32, epochs=10,
               steps_per_epoch=None, validation_steps=None):
    """ Принимаем модель, данные и параметры.
        Возвращаем обученную модель
    """
    features_train, target_train = train_data
    features_test, target_test = test_data
    model.fit(features_train, target_train, 
              validation_data=(features_test, target_test),
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)

    return model

In [15]:
# Обучение модели
model = train_model(create_model((28, 28, 1), optimizer=Adamax(learning_rate=0.007)), 
                    load_train(df_train),
                    load_test(df_test),
                    batch_size=64,
                    epochs=10,
                   )

In [16]:
# Визуализация процесса обучения модели
fig = plt.figure(figsize=(15,5))

ax_1 = fig.add_subplot(1, 2, 1)
ax_1.set_title('Accuracy', fontsize=16)
ax_1.set_xlabel('Epoch Number', fontsize=14)
ax_1.set_ylabel('accuracy', fontsize=14)
ax_1 = plt.plot(model.history.history['acc'], label='training set')
ax_1 = plt.plot(model.history.history['val_acc'], label='testing set')
plt.legend();

# 
ax_2 = fig.add_subplot(1, 2, 2)
ax_2.set_title('Loos function', fontsize=16)
ax_2.set_xlabel('Epoch Number', fontsize=14)
ax_2.set_ylabel('sparse_categorical_crossentropy', fontsize=14)
ax_2 = plt.plot(model.history.history['loss'], label='training set')
ax_2 = plt.plot(model.history.history['val_loss'], label='testing set')


plt.legend();

**Вывод**

Была поставлена задача классификации предметов гардероба по изображению.

Анализ данных выявил, что больших проблем, влияющих на возможность построения модели в наборе данных нет. Были удалены дубликаты, данные были приведены к двумерному виду и нормализованы.

Затем была создана модель на основе нейросетевой архитектуры LeNet, отличающейся от оригинальной тем, что активаторы были заменены на 'relu'. 

Модель показала точность классификации = 0.91.